<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span><ul class="toc-item"><li><span><a href="#Text-Fabric" data-toc-modified-id="Text-Fabric-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Text-Fabric</a></span></li><li><span><a href="#Cuneiform-tablets-in-ATF" data-toc-modified-id="Cuneiform-tablets-in-ATF-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Cuneiform tablets in ATF</a></span><ul class="toc-item"><li><span><a href="#Provenance" data-toc-modified-id="Provenance-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Provenance</a></span></li></ul></li><li><span><a href="#Why-Text-Fabric?" data-toc-modified-id="Why-Text-Fabric?-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Why Text-Fabric?</a></span></li></ul></li><li><span><a href="#Overview" data-toc-modified-id="Overview-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Overview</a></span></li><li><span><a href="#More-information" data-toc-modified-id="More-information-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>More information</a></span></li><li><span><a href="#Installing-Text-Fabric" data-toc-modified-id="Installing-Text-Fabric-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Installing Text-Fabric</a></span><ul class="toc-item"><li><span><a href="#Prerequisites" data-toc-modified-id="Prerequisites-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Prerequisites</a></span></li><li><span><a href="#TF-itself" data-toc-modified-id="TF-itself-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>TF itself</a></span></li><li><span><a href="#Get-the-data" data-toc-modified-id="Get-the-data-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Get the data</a></span><ul class="toc-item"><li><span><a href="#Tip" data-toc-modified-id="Tip-4.3.1"><span class="toc-item-num">4.3.1&nbsp;&nbsp;</span>Tip</a></span></li></ul></li></ul></li><li><span><a href="#Load-Data" data-toc-modified-id="Load-Data-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Load Data</a></span></li><li><span><a href="#A-tour-around-the-TF-API" data-toc-modified-id="A-tour-around-the-TF-API-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>A tour around the TF API</a></span><ul class="toc-item"><li><span><a href="#Sections:-tablet---column---line" data-toc-modified-id="Sections:-tablet---column---line-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Sections: tablet - column - line</a></span></li><li><span><a href="#Navigation" data-toc-modified-id="Navigation-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Navigation</a></span><ul class="toc-item"><li><span><a href="#Source-lines" data-toc-modified-id="Source-lines-6.2.1"><span class="toc-item-num">6.2.1&nbsp;&nbsp;</span>Source lines</a></span></li><li><span><a href="#Source-lines:-refined" data-toc-modified-id="Source-lines:-refined-6.2.2"><span class="toc-item-num">6.2.2&nbsp;&nbsp;</span>Source lines: refined</a></span></li><li><span><a href="#Lines-and-cases" data-toc-modified-id="Lines-and-cases-6.2.3"><span class="toc-item-num">6.2.3&nbsp;&nbsp;</span>Lines and cases</a></span></li><li><span><a href="#Line-numbers" data-toc-modified-id="Line-numbers-6.2.4"><span class="toc-item-num">6.2.4&nbsp;&nbsp;</span>Line numbers</a></span></li></ul></li><li><span><a href="#Deep-cases" data-toc-modified-id="Deep-cases-6.3"><span class="toc-item-num">6.3&nbsp;&nbsp;</span>Deep cases</a></span><ul class="toc-item"><li><span><a href="#Measuring-depth" data-toc-modified-id="Measuring-depth-6.3.1"><span class="toc-item-num">6.3.1&nbsp;&nbsp;</span>Measuring depth</a></span></li></ul></li></ul></li><li><span><a href="#Mass-operations" data-toc-modified-id="Mass-operations-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Mass operations</a></span><ul class="toc-item"><li><span><a href="#Walking-all-nodes" data-toc-modified-id="Walking-all-nodes-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>Walking all nodes</a></span></li><li><span><a href="#All-signs" data-toc-modified-id="All-signs-7.2"><span class="toc-item-num">7.2&nbsp;&nbsp;</span>All signs</a></span></li><li><span><a href="#Particular-signs" data-toc-modified-id="Particular-signs-7.3"><span class="toc-item-num">7.3&nbsp;&nbsp;</span>Particular signs</a></span></li><li><span><a href="#From-instance-to-context" data-toc-modified-id="From-instance-to-context-7.4"><span class="toc-item-num">7.4&nbsp;&nbsp;</span>From instance to context</a></span></li><li><span><a href="#Frequency-list" data-toc-modified-id="Frequency-list-7.5"><span class="toc-item-num">7.5&nbsp;&nbsp;</span>Frequency list</a></span></li></ul></li><li><span><a href="#Structures" data-toc-modified-id="Structures-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Structures</a></span><ul class="toc-item"><li><span><a href="#Quads" data-toc-modified-id="Quads-8.1"><span class="toc-item-num">8.1&nbsp;&nbsp;</span>Quads</a></span><ul class="toc-item"><li><span><a href="#Getting-a-big-quad" data-toc-modified-id="Getting-a-big-quad-8.1.1"><span class="toc-item-num">8.1.1&nbsp;&nbsp;</span>Getting a big quad</a></span></li><li><span><a href="#Unraveling-a-quad" data-toc-modified-id="Unraveling-a-quad-8.1.2"><span class="toc-item-num">8.1.2&nbsp;&nbsp;</span>Unraveling a quad</a></span></li><li><span><a href="#Distribution-by-depth" data-toc-modified-id="Distribution-by-depth-8.1.3"><span class="toc-item-num">8.1.3&nbsp;&nbsp;</span>Distribution by depth</a></span></li></ul></li><li><span><a href="#Clusters" data-toc-modified-id="Clusters-8.2"><span class="toc-item-num">8.2&nbsp;&nbsp;</span>Clusters</a></span><ul class="toc-item"><li><span><a href="#Kinds-of-clusters" data-toc-modified-id="Kinds-of-clusters-8.2.1"><span class="toc-item-num">8.2.1&nbsp;&nbsp;</span>Kinds of clusters</a></span></li><li><span><a href="#Lengths-of-clusters" data-toc-modified-id="Lengths-of-clusters-8.2.2"><span class="toc-item-num">8.2.2&nbsp;&nbsp;</span>Lengths of clusters</a></span></li></ul></li></ul></li><li><span><a href="#Signs" data-toc-modified-id="Signs-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Signs</a></span><ul class="toc-item"><li><span><a href="#Writing-results-to-file" data-toc-modified-id="Writing-results-to-file-9.1"><span class="toc-item-num">9.1&nbsp;&nbsp;</span>Writing results to file</a></span></li><li><span><a href="#Prime" data-toc-modified-id="Prime-9.2"><span class="toc-item-num">9.2&nbsp;&nbsp;</span>Prime</a></span></li><li><span><a href="#Variant" data-toc-modified-id="Variant-9.3"><span class="toc-item-num">9.3&nbsp;&nbsp;</span>Variant</a></span></li><li><span><a href="#Modifier" data-toc-modified-id="Modifier-9.4"><span class="toc-item-num">9.4&nbsp;&nbsp;</span>Modifier</a></span></li><li><span><a href="#Full-grapheme-overview" data-toc-modified-id="Full-grapheme-overview-9.5"><span class="toc-item-num">9.5&nbsp;&nbsp;</span>Full grapheme overview</a></span></li></ul></li><li><span><a href="#Representation" data-toc-modified-id="Representation-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Representation</a></span></li><li><span><a href="#Collation" data-toc-modified-id="Collation-11"><span class="toc-item-num">11&nbsp;&nbsp;</span>Collation</a></span><ul class="toc-item"><li><span><a href="#TF-ad" data-toc-modified-id="TF-ad-11.1"><span class="toc-item-num">11.1&nbsp;&nbsp;</span>TF ad</a></span></li><li><span><a href="#Back-to-collation" data-toc-modified-id="Back-to-collation-11.2"><span class="toc-item-num">11.2&nbsp;&nbsp;</span>Back to collation</a></span></li><li><span><a href="#Collect-sign-pairs" data-toc-modified-id="Collect-sign-pairs-11.3"><span class="toc-item-num">11.3&nbsp;&nbsp;</span>Collect sign-pairs</a></span></li></ul></li></ul></div>

<img align="right" src="images/tf-small.png"/>


# Tutorial

This notebook gets you started with using
[Text-Fabric](https://github.com/Dans-labs/text-fabric) for coding in cuneiform tablet transcriptions.

## Introduction

### Text-Fabric

Text-Fabric is a model for textual data with annotations that is optimized for efficient data analysis. Not only that, it also facilitates the creation of new, derived data, which can be added to the original data.
Data combination is a feature of Text-Fabric.

Text-Fabric is being used for the
[Hebrew Bible](https://github.com/ETCBC/bhsa)
and a large body of linguisitic annotations on top of it. The researchers of the 
[ETCBC](http://etcbc.nl) thought that a plain database is not a satisfactory text model,
and that XML is too limited too express multiple hierarchies in a text smoothly.

That's why they adopted a model by
[Doedens](http://books.google.nl/books?id=9ggOBRz1dO4C)
that reflects more of the essential properties of text (sequence, embedding). This model is the basis of MQL, a working text-database system.
Text-Fabric is based on the same
[model](https://github.com/Dans-labs/text-fabric/wiki/Data-model),
and once the data is in Text-Fabric, it can be exported to MQL.

See more on the effort of modeling the Hebrew Bible in Dirk's article
[The Hebrew Bible as Data: Laboratory - Sharing - Experiences](https://doi.org/10.5334/bbi.18)

With data in Text-Fabric, it becomes possible to build rich online interfaces on the data of ancient texts.
For the Hebrew Bible, we have built
[SHEBANQ](https://shebanq.ancient-data.org).

Working with TF is a bit like buying from IKEA. You get your product in bits and pieces, and you assemble it yourself. TF decomposes any dataset into its components, nicely stacked per component, with every component uniquely labeled. You go to the store, make your selection, enter the warehouse, collect your parts, and, at home, assemble your product.

In order to enjoy an IKEA product, you do not need to be a craftsman, but you do need to be able to handle a screw driver.

In the TF world, it is the same. You do not have to be a professional programmer, but you do need to be able to program little things. A first course in Python is enough.

Another parallel: in IKEA you take a package with components home, and there you assemble it. 
In TF it is likewise: you download the TF data, and then you write a little program. Inside that program you can call up the Text-Fabric tool, which act as the IKEA user manual. But your program takes control, not Text-Fabric.

The best environment to enjoy Text-Fabric is in Python programs that you develop in a 
[Jupyter Notebook](http://jupyter.readthedocs.io/en/latest/).
This tutorial is such a notebook. If you are reading it online, you see text bits and code bits,
but you cannot execute the code bits.

If you download this tutorial, and you have installed Python, Jupyter, and Text-Fabric,
you can *execute* the code bits.

### Cuneiform tablets in ATF

Cuneiform tablets have been transcribed in ATF files, in which the marks on a tablet are represented
by ascii characters. The marks on a tablet have structure (they can be composed, they build cases and lines) and properties (they can be uncertain).

<img align="left" src="images/P005381-obverse-photo.png" width="25%"/>
<img align="left" src="images/P005381-obverse-lineart.png" width="25%"/>
```
&P005381 = MSVO 3, 70
#version: 0.1
#atf: lang qpc
```
<img align="right" src="images/P005381-obverse-atf.png" width="50%"/>

<img align="left" src="images/P005381-reverse-photo.png" width="25%"/>
<img align="left" src="images/P005381-reverse-lineart.png" width="25%"/>
<img align="right" src="images/P005381-reverse-atf.png" width="50%"/>

*Obverse* and *reverse* face of tablet
[P005381](https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P005381). 

>Authors: Robert K. Englund and Peter Damerow

>Transcription by Robert K. Englund

>UCLA Library ARK	21198/zz001q057f

#### Provenance

We have taken a corpus of ca. 6000 tablets, from the
[Uruk-III/IV period](http://cdli.ox.ac.uk/wiki/doku.php?id=proto-cuneiform)
(4000-3100 BC).
These tablets are *proto-cuneiform*, 

We have downloaded the transcriptions from the 
**Cuneiform Digital Library Initiative**
[CDLI](https://cdli.ucla.edu),
and converted them to Text-Fabric.

### Why Text-Fabric?
When you search for tablet data in an ATF file, you can do so conveniently by using regular expressions.

However, the ATF transcriptions have become packed with information. Not every transcriber uses ATF in the same way, and there are a few coding errors in the sources.

That means that the most obvious search expressions will leave out cases. Either you live with that, or you refine your search expressions.

An other issue is, that when you look for something, your search expressions must reflect the shape of not
only your target, but also everything else. There is virtually no separation of concerns.

When your search task becomes complicated, your mind tends to get overloaded with the details of the transcription format, rather than with the quest you would like to have in mind.

You also want to be able to divide a research task into smaller chunks.
Whereas regular expressions are a wonderful tool to look for patterns, on their own they fall short
when your search involves nested structures, intermediate sets of patterns, special functions.



## Overview

* we tell you how to get Text-Fabric on your system;
* we tell you how to get a set of cuneiform tablet transcriptions on your system;
* we make a tour, showing how to use the data;
* we add a real world exercise in collocation.

## More information
Chances are that a bit of reading about the underlying
[data model](https://github.com/Dans-labs/text-fabric/wiki/Data-model)
helps you to follow the exercises below, and vice versa.

We have checked the conversion from the transcriptions to Text-Fabric extensively.
Cruelly, you might say. You can follow the checks
in a separate notebook [checks](checks.ipynb).

It will not harm to open the following reference pages in separate browser tabs.

[Feature docs](https://github.com/Dans-labs/Nino-cunei/blob/master/docs/transcription.md)

[Cunei API](https://github.com/Dans-labs/Nino-cunei/blob/master/docs/cunei.md)

[Utils API](https://github.com/Dans-labs/Nino-cunei/blob/master/docs/utils.md)

[Text-Fabric API](https://github.com/Dans-labs/text-fabric)

## Installing Text-Fabric

### Prerequisites

You need to have Python on your system. Most systems have it out of the box,
but alas, that is python2 and we need at least python 3.6.

Install it from [python.org]() or from [Anaconda]().
If you got it from python.org, you also have to install [Jupyter]().

### TF itself

```
pip install text-fabric
```

if you have installed Python with the help of Anaconda, or

```
sudo -H pip3 install text-fabric
```
if you have installed Python from [python.org](https://www.python.org).

###### Execute: If all this is done, the following cells can be executed.

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys, os, collections
from tf.fabric import Fabric

### Get the data

You can get all data needed for this tutorial
from GitHub as follows.

We suggest you make an appropriate directory in your home directory:

```
github/Dans-labs
```

then go to that directory in a terminal, and then say

```
git clone https://github.com/Dans-labs/Nino-cunei
```

After that your directory structure shold look like this:

    your home direcectory\
    |                     - github\
    |                       |      - Dans-labs\
    |                       |        |         - Nino-cunei
    
#### Tip
If you start computing with this tutorial, first copy its parent directory to somewhere else,
outside your `Nino-cunei` directory.
If you pull changes from the `Nino-cunei` repository later, your work will not be overwritten.
Where you put your tutorial directory is up till you.
It will work from any directory.

###### Execute: it this has been done, you can execute the next cells

In [4]:
REPO = '~/github/Dans-labs/Nino-cunei'
SOURCE = 'uruk'
VERSION = '0.1'
CORPUS = f'{REPO}/tf/{SOURCE}/{VERSION}'
SOURCE_DIR = os.path.expanduser(f'{REPO}/sources/cdli')
PROGRAM_DIR = os.path.expanduser(f'{REPO}/programs')
TEMP_DIR = os.path.expanduser(f'{REPO}/_temp')
REPORT_DIR = os.path.expanduser(f'{REPO}/reports')

In [5]:
sys.path.append(PROGRAM_DIR)
from cunei import Cunei

In [6]:
for cdir in (TEMP_DIR, REPORT_DIR):
    os.makedirs(cdir, exist_ok=True)

In [7]:
TF = Fabric(locations=[CORPUS], modules=[''], silent=False )

This is Text-Fabric 3.2.1
Api reference : https://github.com/Dans-labs/text-fabric/wiki/Api
Tutorial      : https://github.com/Dans-labs/text-fabric/blob/master/docs/tutorial.ipynb
Example data  : https://github.com/Dans-labs/text-fabric-data

32 features found and 0 ignored


## Load Data

The transcriptions of the tablets in their TF form is organized in a model of nodes, edges and features.

The things such as tablets, faces, columns, cases, and, at the most basic level, signs, are numbered.
The signs correspond to number 1 ... ca. 120,000, in the same order as they occur in the corpus.
All other things are built from signs. They have number from ca 120,000 to 450,000.

In TF, we call these numbers *nodes*. Like a barcode, this number gives access to a whole bunch of
information about the corresponding object.

For example, lines have a property (in TF we call it a *feature*) called `fullNumber`. 
It contains the hierarchical number found at the start of the line in the transcription.

If the node for a line is `n`, we can find its hierarchical number by saying

```
F.fullNumber.v(n)
```

In words, it reads as:

* `F`: I want to look up a `F`eature
* `fullNumber`: the name of the feature
* `.v`: I want the value of that feature
* `(n)`: for the given node `n`

Seen in this way, the data is like a gigantic spreadsheet of 450,000 rows (the nodes),
with 30 columns (the features).

There is a bit more to it, since the nodes can be grouped together in ways we will see later on.

###### Execute: the next cell loads most features

In [8]:
api = TF.load('''
    grapheme prime repeat
    variant variantOuter
    modifier modifierInner modifierFirst
    damage uncertain remarkable written
    period name type identifier catalogId
    number fullNumber origNumber badNumbering
    crossref text
    srcLn srcLnNum
    op sub comments''')
api.makeAvailableIn(globals())
CUNEI = Cunei(api)

  0.00s loading features ...
   |     0.00s B catalogId            from /Users/dirk/github/Dans-labs/Nino-cunei/tf/uruk/0.1
   |     0.02s B fullNumber           from /Users/dirk/github/Dans-labs/Nino-cunei/tf/uruk/0.1
   |     0.01s B number               from /Users/dirk/github/Dans-labs/Nino-cunei/tf/uruk/0.1
   |     0.07s B grapheme             from /Users/dirk/github/Dans-labs/Nino-cunei/tf/uruk/0.1
   |     0.05s B srcLn                from /Users/dirk/github/Dans-labs/Nino-cunei/tf/uruk/0.1
   |     0.03s B srcLnNum             from /Users/dirk/github/Dans-labs/Nino-cunei/tf/uruk/0.1
   |     0.00s B prime                from /Users/dirk/github/Dans-labs/Nino-cunei/tf/uruk/0.1
   |     0.01s B repeat               from /Users/dirk/github/Dans-labs/Nino-cunei/tf/uruk/0.1
   |     0.01s B variant              from /Users/dirk/github/Dans-labs/Nino-cunei/tf/uruk/0.1
   |     0.00s B variantOuter         from /Users/dirk/github/Dans-labs/Nino-cunei/tf/uruk/0.1
   |     0.00s B modi

## A tour around the TF API

We have the data of over 6000 tablets at our fingertips now. But how can we see it?
In a meaningful way?

The following tour shows you the most important parts of the Text-Fabric API.
It is a set of functions, defined in the TF package, that you can use in your own program.

This notebook itself is our current program, and we have just loaded the API.

See in the next cells how we navigate to the tablets we want and pick our data from them.

The full API-reference documentation is available on the
[Text-Fabric wiki](https://github.com/Dans-labs/text-fabric/wiki/Api).

We also have collected frequently used functionality for this corpus in the
[Cunei API](https://github.com/Dans-labs/Nino-cunei/blob/master/docs/cunei.md).

### Sections: tablet - column - line

Let's start with just a single, specific tablet P448702.

Here is its transcription, copied from the CDLI source.

```
&P448702 = www archaeo-auction 004 
#atf: lang qpc 
@tablet 
@obverse 
@column 1 
$ beginning broken 
1'. [n] , [...] KA~a 
2.a'. [n] 2(N14) 3(N01) , KASZ~b NUN~a 
2.b'. 3(N01) , KASZ~a? GI 
3'. [n] 6(N14)#? 4(N01) 2(N39~a) , X [...] 
@column 2 
$ beginning broken 
$ blank space 
1'. , U4 |U4x1(N01)| SUKUD@inversum? NA 
@column 3 
$ beginning broken 
$ blank space 
@reverse 
$ (not imaged) 
```

There is a node for this tablet. How do we find it?

The easiest way is to ask TF to find the node on the basis of a section specification.

When we converted the transcriptions to TF, we made it so that tablets are the first
section level, columns the second, and lines the third. 

In order to find the node corresponding to a section, we have to give a tuple
`(tabletName, columnNumber, lineNumber)` to the appropriate function in TF.

Here you may leave out `lineNumber` (to specify a column), or both `columnNumber` and `lineNumber` 
(to specify a tablet).
The latter is our case here.

In [9]:
justATablet = T.nodeFromSection(('P448702',))
justATablet

150865

Ok, this is the node, the barcode, as it were. It gives you access to all data we have stored of this
tablet.

One of the things we have stored for tablets, columns, and lines, is the original transcription line.
Let's retrieve it.

In [10]:
F.srcLn.v(justATablet)

'&P448702 = www archaeo-auction 004 '

This is the source line of the tablet header.
How do we get the other source lines?

### Navigation

We'll need another concept first.
Things on a tablet have spatial relationships. 
While the transcriptions abstract from the physical relationships, there are still a few
important ones left: sequence and embedding.

At the most basic level we have signs. 
For all other levels, TF knows exactly which signs are part of the objects of those levels.

And based on that knowledge, we can travel from objects to the objects embedded in it, and back,
and to the right and left siblings.

All these functions start with `L.`, and they take a starting node as argument.

* `L.d()` goes "down": from enbedder to embeddee;
* `L.u()` goes "up": from embeddee to embedder;
* `L.p()` goes "previous": to the first left sibling;
* `L.n()` goes "next": to the first right sibling.

We can now find all the nodes contained in the example tablet.
When we retrieve them, we also fetch their *type*: is it a face, a column, a sign?
The type of a node is stored in the feature `otype`. 
This feature is so central to the TF model that every TF dataset has it.

In [11]:
for child in L.d(justATablet):
    print(f'{child:>6} of type {F.otype.v(child)}')

180739 of type comment
    56 of type sign
157262 of type face
166709 of type column
180740 of type comment
    57 of type sign
229518 of type line
266443 of type case
196542 of type cluster
    58 of type sign
196543 of type cluster
    59 of type sign
    60 of type sign
229519 of type line
266444 of type case
266445 of type case
196544 of type cluster
    61 of type sign
    62 of type sign
    63 of type sign
    64 of type sign
    65 of type sign
266446 of type case
266447 of type case
    66 of type sign
    67 of type sign
    68 of type sign
229520 of type line
266448 of type case
196545 of type cluster
    69 of type sign
    70 of type sign
    71 of type sign
    72 of type sign
    73 of type sign
196546 of type cluster
    74 of type sign
166710 of type column
180741 of type comment
    75 of type sign
180742 of type comment
    76 of type sign
229521 of type line
266449 of type case
    77 of type sign
146957 of type quad
    78 of type sign
    79 of type sign
    80 of

A few things to note:

* the numbers are not really important to us, only to the program;
  soon, we will stop showing them altogether;
* the order in which the nodes are delivered, is a natural one:
  1. first things first, and 
  2. bigger things before the smaller things they contain.

#### Source lines

So in order to find all the source lines of this tablet, we can just take all these nodes,
see whether they have a value for feature *srcLn*, retrieve it, an print it.

Let's do it, and write this as a function, i.e. a recipe that we can apply to every tablet.

In [12]:
def getSource(tablet):
    sourceLines = []
    sourceLines.append(F.srcLn.v(tablet))
    for child in L.d(tablet):
        sourceLine = F.srcLn.v(child)
        if sourceLine:
            sourceLines.append(sourceLine)
    return sourceLines

This function gets all source lines in a list.
Let's apply it to our sample tablet, and print out the list:

In [13]:
sourceLines = getSource(justATablet)
print('\n'.join(sourceLines))

&P448702 = www archaeo-auction 004 
#atf: lang qpc 
@obverse 
@column 1 
$ beginning broken 
1'. [n] , [...] KA~a 
2.a'. [n] 2(N14) 3(N01) , KASZ~b NUN~a 
2.b'. 3(N01) , KASZ~a? GI 
3'. [n] 6(N14)#? 4(N01) 2(N39~a) , X [...] 
@column 2 
$ beginning broken 
$ blank space 
1'. , U4 |U4x1(N01)| SUKUD@inversum? NA 
@column 3 
$ beginning broken 
$ blank space 
@reverse 
$ (not imaged) 


This is nearly the original. The only difference we spot is that we have left out
the `@tablet` line. This has been done because the `&P` line is the true header of the
tablet. In fact, the `@tablet` line is not consistently present in the source.

This is an example where the TF data is a little bit more streamlined than the source transcriptions.

Speaking of which: see that strange modifier on `SUKUD`, the `@inversum`?
It is not in the [Oracc primer on ATF](http://oracc.museum.upenn.edu/doc/help/editinginatf/primer/inlinetutorial/index.html)

Probably it is a typo, and the `inversum` should be replaced by a single letter.
While Dirk was programming, he decided to "correct" this into `@v`, but what really is needed here is
to correct the source. Or we could correct the correction.
In either case, we expect to run the conversion again, with all issues like these resolved.

To see what those issues are, see the complete list of
[diagnostics](https://github.com/Dans-labs/Nino-cunei/blob/master/reports/diagnostics.tsv).

Let's just enjoy the new power, and get the source lines of another, important tablet: 
[P005381](https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P005381).

We show the photos, lineart and transcriptions of both faces of this tablet.
For provenance: see above or follow the link.

<img align="left" src="images/P005381-obverse-photo.png" width="50%"/>
<img align="right" src="images/P005381-reverse-photo.png" width="50%"/>

<img align="left" src="images/P005381-obverse-lineart-annot.png" width="50%"/>
<img align="right" src="images/P005381-reverse-lineart.png" width="50%"/>

In [14]:
exampleTablet = T.nodeFromSection(('P005381',))
sourceLines = getSource(exampleTablet)
print('\n'.join(sourceLines))

&P005381 = MSVO 3, 70
#version: 0.1
#atf: lang qpc
@obverse
@column 1
1.a. 2(N14) , SZE~a SAL TUR3~a NUN~a
1.b. 3(N19) , |GISZ.TE|
2. 1(N14) , NAR NUN~a SIG7
3. 2(N04)# , PIRIG~b1 SIG7 URI3~a NUN~a
@column 2
1. 3(N04) , |GISZ.TE| GAR |SZU2.((HI+1(N57))+(HI+1(N57)))| GI4~a
2. , GU7 AZ SI4~f
@reverse
@column 1
1. 3(N14) , SZE~a
2. 3(N19) 5(N04) ,
3. , GU7
@column 2
1. , AZ SI4~f


By the way, in column 2, line 1 we find a composition of three levels: 
`|SZU2.((HI+1(N57))+(HI+1(N57)))|`

We call compositions of signs by means of operators such as `.` and `x` and `+` *quads*.
They are also a node type, the first node type above the "atomic" sign.

We'll see a lot of *quads* later on.

#### Source lines: refined

If you just want the lines, and nothing else in between, you can instruct the `L.d` function to
let through only nodes of a certain type.

Let's improve the `getSource` function to give it an additional, optional argument that
tells us the kind of object we want to see.

In [15]:
def getSource(tablet, nodeType=None):
    sourceLines = []
    sourceLines.append(F.srcLn.v(tablet))
    for child in L.d(tablet, nodeType):
        sourceLine = F.srcLn.v(child)
        if sourceLine:
            sourceLines.append(sourceLine)
    return sourceLines

If we call it without node type, we get the same results.

Let's call it with `nodeType='face'`:

In [16]:
sourceLines = getSource(exampleTablet, nodeType='face')
print('\n'.join(sourceLines))

&P005381 = MSVO 3, 70
@obverse
@reverse


We see only the face specifiers.

Now let's grab the lines: 

In [17]:
sourceLines = getSource(exampleTablet, nodeType='case')
print('\n'.join(sourceLines))

&P005381 = MSVO 3, 70
1.a. 2(N14) , SZE~a SAL TUR3~a NUN~a
1.b. 3(N19) , |GISZ.TE|
2. 1(N14) , NAR NUN~a SIG7
3. 2(N04)# , PIRIG~b1 SIG7 URI3~a NUN~a
1. 3(N04) , |GISZ.TE| GAR |SZU2.((HI+1(N57))+(HI+1(N57)))| GI4~a
2. , GU7 AZ SI4~f
1. 3(N14) , SZE~a
2. 3(N19) 5(N04) ,
3. , GU7
1. , AZ SI4~f


#### Lines and cases

Note that we actually supplied `nodeType='case'`.
This is because we have modelled the data in such a way, that a line corresponds to all
transcription lines in a column that start with the same number.
Lines are divided in cases and sub-cases according to the hierarchical line numbers.
At the lowest level of sub-case we encounter the material that belong to single
transcription lines.

Also when lines only have a single case, we will have a case embedded in the line.

So *case* is the node type for the geometric divisions of lines,
coded by the hierarchical line number.

#### Line numbers

The TF data also retains the original line numbers, i.e. the line number in the file
that contains the transcriptions.
We do not mean the hierarchical line numbers of the cases!
Let us once again improve the `getSource` function by 
giving it an optional argument for fetching line numbers too:

In [18]:
def getSource(tablet, nodeType=None, lineNumbers=False):
    sourceLines = []
    lineNumber = ''
    if lineNumbers:
        lineNumber = f'{F.srcLnNum.v(tablet):>5}: '
    sourceLines.append(f'{lineNumber}{F.srcLn.v(tablet)}')
    for child in L.d(tablet, nodeType):
        sourceLine = F.srcLn.v(child)
        lineNumber = ''
        if sourceLine:
            if lineNumbers:
                lineNumber = f'{F.srcLnNum.v(child):>5}: '
            sourceLines.append(f'{lineNumber}{sourceLine}')
    return sourceLines

Now we can print the lines with source line numbers, so that we can look them up in the
source easily:

In [19]:
sourceLines = getSource(exampleTablet, nodeType='case', lineNumbers=True)
print('\n'.join(sourceLines))

80927: &P005381 = MSVO 3, 70
80932: 1.a. 2(N14) , SZE~a SAL TUR3~a NUN~a
80933: 1.b. 3(N19) , |GISZ.TE|
80934: 2. 1(N14) , NAR NUN~a SIG7
80935: 3. 2(N04)# , PIRIG~b1 SIG7 URI3~a NUN~a
80937: 1. 3(N04) , |GISZ.TE| GAR |SZU2.((HI+1(N57))+(HI+1(N57)))| GI4~a
80938: 2. , GU7 AZ SI4~f
80941: 1. 3(N14) , SZE~a
80942: 2. 3(N19) 5(N04) ,
80943: 3. , GU7
80945: 1. , AZ SI4~f


You'll appreciate the way you can build up your own power tools as you go.
The functions of TF provide a foundation, so that start when all the gory details
have been taken care of.

### Deep cases

How deep can cases go?
We explore the distribution of cases with respect to their depth.

#### Measuring depth

Cases are structures that can be nested.
It is interesting to know how deep cases can be nested.

We need a function that computes the depth of a case.
We program that function in such a way that it also works for *quads*,
an other nested structure.

The idea of this function is:
* if a structure does not have sub-structures, its depth is 1;
* the depth of a structure is 1 more than the maximum of the depths of its sub-structures.

In [20]:
def depthStructure(node, nodeType):
    subDepths = [
        depthStructure(subNode, nodeType) 
        for subNode in E.sub.f(node) 
        if F.otype.v(subNode) == nodeType
    ]
    if len(subDepths) == 0:
        return 1
    else:
        return max(subDepths) + 1

We are going to collect all cases in buckets according to their depths.

In [21]:
caseDepths = collections.defaultdict(list)

for case in F.otype.s('case'):
    caseDepths[depthStructure(case, 'case')].append(case)

caseDepthsSorted = sorted(
    caseDepths.items(), 
    key=lambda x: (-x[0], -len(x[1])),
)

for (depth, cases) in caseDepthsSorted:
    print(f'{len(cases):>5} cases with depth {depth}')

   23 cases with depth 5
   80 cases with depth 4
 1096 cases with depth 3
 8316 cases with depth 2
42170 cases with depth 1


We show where the deepest cases are.
We also retrieve the size of the face containing that case.

In [22]:
deepestCases = caseDepthsSorted[0][1]

deepCaseData = []

for deepCase in deepestCases:
    (tNumber, colNumber, lineNumber) = T.sectionFromNode(deepCase)
    face = L.u(deepCase, otype='face')[0]
    faceSlots = L.d(face, otype='sign')
    lengthFace = faceSlots[-1] - faceSlots[0]
    deepCaseData.append((tNumber, colNumber, lengthFace))
    
deepCaseDataSorted = sorted(deepCaseData, key=lambda x: (x[2], x[1]))

for (tNumber, colNumber, lengthFace) in deepCaseDataSorted:
    print(f'{tNumber}:{colNumber} ({lengthFace:>3} signs on face)')

P218054:reverse:1 ( 18 signs on face)
P006092:obverse:1 ( 22 signs on face)
P002694:reverse:1 ( 35 signs on face)
P325754:reverse:1 ( 38 signs on face)
P006295:reverse:1 ( 40 signs on face)
P004735:obverse:2 ( 42 signs on face)
P004735:obverse:2 ( 42 signs on face)
P002856:obverse:2 ( 45 signs on face)
P005322:reverse:1 ( 47 signs on face)
P411608:obverse:1 ( 48 signs on face)
P003531:obverse:1 ( 54 signs on face)
P325234:reverse:1 ( 54 signs on face)
P006160:obverse:1 ( 59 signs on face)
P006307:reverse:1 ( 63 signs on face)
P003529:obverse:1 ( 99 signs on face)
P003529:obverse:2 ( 99 signs on face)
P387752:obverse:0 (112 signs on face)
P387752:obverse:0 (112 signs on face)
P006056:reverse:1 (112 signs on face)
P003808:obverse:3 (307 signs on face)
P003808:obverse:5 (307 signs on face)
P003808:obverse:6 (307 signs on face)
P003808:obverse:6 (307 signs on face)


It seems that the first entry (on P218054) and the last four ones
on (P003808) are on remarkable faces.
The first one is a deep case on a very small face, and the last four ones are
deep cases all on the same face.

We show the sources of those tablets. First the small tablet, note the `@reverse` face.

In [23]:
smallTablet = T.nodeFromSection(('P218054',))
sourceLines = getSource(smallTablet)
print('\n'.join(sourceLines))

&P218054 = CDLB nn, CMAA 020-C0003
#version: 0.1
#atf: lang qpc
@obverse
@column 1
1. [...] 1(N01)# , [...] X X
2.a. 6(N01) , PAP~a DUR~b
2.b. 2(N01) ,
3.a1A. 3(N01) , DUR~b
3.a1B. 1(N01) , MASZ2
3.a2. , NI~a 3(N57)
3.b. 6(N01) , GI
4. 1(N01) , MASZ2 ZATU628~a DIM~a SAL
5. 1(N01) , NUN~b DIM~a
6. 1(N01) , NUN~b GAR SAG ZATU694~c
7.a. 3(N01) , DUR~b UD5~a
7.b. 1(N01) , GI
8.a. 2(N01)# [...] , X [...]
8.b. 5(N01) , GI
9. [...] , [...]
@column 2
1. [...] 6(N01)# , [...]
2.a. 3(N01) , SI TE NIM~b2
2.b. 1(N01) ,
3.a. 3(N01) , ZATU843
3.b. 3(N01) ,
4.a. 2(N01) , NA~a GI KU6~a
4.b1. 1(N01) ,
4.b2. 1(N01) , DUR~b
@reverse
@column 1
1.a1A1. [...] 5(N01)# , [...] UDU~a#?
1.a1A2. [...] 7(N01)# , MASZ2
1.a1B. 4(N14) 1(N01) , DUR~b
1.a2A. [...] , [...]
1.a2B. 2(N14) 3(N01) , UDU~a GI
1.b. , |LAL2~axNIM~b2|# [...]


<img align="left" src="images/P218054-reverse-photo.png" width="75%"/>

> From [CDLI](https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P218054)

Below we do the big tablet, note the `@obverse` face.

In [24]:
bigTablet = T.nodeFromSection(('P003808',))
sourceLines = getSource(bigTablet)
print('\n'.join(sourceLines))

&P003808 = IM —
#version: 0.1
#atf: lang qpc
@obverse
@column 1
1. [...] , [...] MUSZ3~a# KASZ~b EZEN~b
2. 1(N01) , SAL AMA~a
3. 4(N01) , MEN~a NUNUZ~c DU
4. 1(N01) , PAP~a GA2~a1 IB~a
@column 2
1.a. [...] , [...]
1.b. [...] , [...]
2.a. [...] , [...]
2.b. [...] , KAB ZI~a
3.a. [...] , [...] X
3.b. , SZUBUR BU~a SUKKAL
4.a. 2(N01)# [...] , X KUR~a SIG2~b SZE~a
4.b. , GESZTU~b# [...]
5.a. 6(N01) , PAD~a GA~a
5.b1. 3(N57) , |SZE~a&SZE~a| DU
5.b2. 3(N57) , DU AB2 SAG#
@column 3
1.a. [...] , [...] DUG~c#
1.b1. [...] 1(N01) , GAN~c [...]
1.b2A. 3(N57) , EN~a BAD AB~a@g
1.b2B. 1(N57) , PAP~a SIG7 NUN~a GUL
1.c. [...] , [...] IB~a#
2.a. 1(N01) , DUG~c
2.b1A. [...] , |DUG~cx1(N57)|
2.b1B1. [...] , GAN~c SZU
2.b1B2. , SZITA~a1 ADAB UTUL~a
2.b2A. 2(N01) , SILA3~a GARA2~a
2.b2B1. 1(N57) , SZA SZU
2.b2B2. 1(N57) , ZATU659? PAP~a SUKKAL
2.b3. 4(N01) , |SILA3~axGA~a| SZA
2.c. , SUG5 LA2 IB~a
3.a1. [...] , X AMA~a AN [...]
3.a2. 1(N01) , KI# NA2~a [...]
3.a3. 1(N01) , MIR~b |ZATU651xNUN~a|
3.b. , GES

<img align="left" src="images/P003808-obverse-photo.png" width="100%"/>

> From [CDLI](https://cdli.ucla.edu/search/search_results.php?SearchMode=Text&ObjectID=P003808)

> This is the [detail image](https://cdli.ucla.edu/dl/photo/P003808_d.jpg)

## Mass operations

We have a mass of data, and we want to do meaningful things with that mass.
Here are a few ways to address all nodes, and significant subsets of all nodes.

### Walking all nodes

The generator `N()` provides you with all nodes, one by one, in a sequence.
They come in the natural order, as we have seen above:

* first things first
* bigger things before the smaller things they contain.

Let's print the first 20 nodes, together with their types.

In [25]:
LIMIT = 20

times = 0
for n in N():
    print(f'{n:>6} : {F.otype.v(n)}')
    times += 1
    if times >= 20:
        break

150862 : tablet
180733 : comment
     1 : sign
180734 : comment
     2 : sign
157258 : face
166700 : column
229498 : line
266423 : case
196539 : cluster
     3 : sign
     4 : sign
     5 : sign
166701 : column
229499 : line
266424 : case
     6 : sign
     7 : sign
     8 : sign
196540 : cluster


### All signs

Suppose we are only interested in the signs, not how they are organized in cases, lines,
columns, faces and tablets.

Suppose we just want to count them.

We could use `N()` as follows:

In [26]:
times = 0
for n in N():
    if F.otype.v(n) != 'sign':
        continue
    times += 1
print(times)

146955


In fact, TF makes it a bit easier to get to all sign nodes, or nodes from whatever type.

We have already used the `F.feature.v(n)` idiom.
But you can do more with features, beyond fetching particular values.

You can also retrieve the sequence of nodes that support a particular value.

We want to retrieve all nodes that support the value `sign` in their `otype` feature.

Here we go, and note that we use `.s(value)` instead of `.v(node)`.

In [27]:
signs = F.otype.s('sign')
len(signs)

146955

Signs have a feature `grapheme`, filled with the name of the sign, without variants (allographs), modifiers, and flags. Let's see the first 20 graphemes.

In [28]:
for sign in signs[0:20]:
    print(f'{sign:>2} : {F.grapheme.v(sign)}')

 1 : 
 2 : 
 3 : …
 4 : X
 5 : X
 6 : N14
 7 : X
 8 : SANGA
 9 : …
10 : 
11 : 
12 : X
13 : X
14 : X
15 : X
16 : X
17 : X
18 : X
19 : X
20 : X


A few things to note:

* there are empty signs
* there is the sign `…`

During conversion, we have put empty signs in metadata lines and rulings.
That is the mechanism by which TF knows where in the corpus they are positioned.
They do not belong to the material that corresponds to tablet content, except maybe in the
case of rulings.

The sign `…` originates from `...` in transcriptions, which is mostly surrounded by
`[ ]`. The brackets signify uncertainty, and can be put around other signs as well,
even sequences of signs. These brackets denote *clusters*, a node type that we will encounter
later.

So, the expression `[...]` in the source translates to a *cluster* in TF, with just one
sign in it, which has *grapheme* value `…`. 
Of course, this is not a real grapheme, and it will be easy to weed out such graphemes if
you do not need to take them into account. 

### Particular signs

Suppose we want to get all occurrences of the grapheme `SANGA`.

We can use the same trick as for getting the `signs`: we ask for the nodes that support
the value `SANGA` in their `grapheme` feature.

In [29]:
sangas = F.grapheme.s('SANGA')
len(sangas)

747

As an example of what you can do with such a set, let's pick only those SANGAs that
do not have a variant. We want `SANGA`, but not `SANGA~a`.

In [30]:
pureSangas = [s for s in sangas if F.variant.v(s) is None]
len(pureSangas)

0

Ah, all sangas are with variant (allograph) in this corpus. Are they all `~a`?

In [31]:
aSangas = [s for s in sangas if F.variant.v(s) == 'a']
len(aSangas)

729

Not all of them, let's see what other variants occur:

In [32]:
sangaVariants = set()
otherSangas = []

for s in sangas:
    if s not in aSangas:
        otherSangas.append(s)
        variant = F.variant.v(s)
        sangaVariants.add(variant)
        print(f'{s:>6} SANGA~{variant}')
print(f'All variants: {", ".join(sorted(sangaVariants))}')

 65752 SANGA~b
122525 SANGA~b
122536 SANGA~b
122577 SANGA~b
122593 SANGA~b
123008 SANGA~b
123011 SANGA~b
123019 SANGA~b
123025 SANGA~b
123052 SANGA~b
124094 SANGA~b
124783 SANGA~b
127407 SANGA~b
133550 SANGA~b
133681 SANGA~c
135991 SANGA~b
138239 SANGA~b
142282 SANGA~c
All variants: b, c


### From instance to context

We want to know where those other SANGA variants are.

Earlier we saw how we could get nodes if we knew the section.
Here we need the opposite. We know a few nodes, and want to get the section.
TF has a function for that:

In [33]:
for s in otherSangas:
    print(T.sectionFromNode(s))

('P273433', 'obverse:2', '3')
('P000884', 'obverse:1', '2')
('P000884', 'obverse:2', '1')
('P000886', 'obverse:2', '2')
('P000888', 'obverse:1', '1')
('P000958', 'obverse:2', '3')
('P000958', 'obverse:2', '4')
('P000958', 'obverse:2', '7')
('P000958', 'obverse:3', '2')
('P000958', 'obverse:5', '2')
('P001078', 'obverse:2', '2')
('P001161', 'obverse:1', '1')
('P001377', 'obverse:3', '3')
('P002079', 'obverse:1', '1')
('P002207', 'obverse:1', '1')
('P003233', 'obverse:2', '2')
('P000121', 'obverse:2', '3')
('P003454', 'obverse:2', '2')


If we want to see the source code line of these sangas, we can do it by what we have learned so far:

In [34]:
for s in otherSangas:
    for case in L.u(s, otype='case'):
        srcLine = F.srcLn.v(case)
        if srcLine is not None:
            print(f'{F.srcLnNum.v(case):>5}: {srcLine}')

53330: 3.b. 2(N14) 6(N01) , SANGA~b? 
 1292: 2. 2(N01) , SANGA~b MA
 1296: 1. 1(N01)# , X SANGA~b#?
 1322: 2. 1(N01)# , ZATU713# SANGA~b#
 1339: 1. [...] 2(N01)# , HASZHUR# SANGA~b# [...]
 1786: 3. 1(N01) , UMUN2# SANGA~b#
 1787: 4. 1(N01) , ZATU678# SANGA~b?
 1790: 7. 1(N01) , SUG5 SANGA~b
 1793: 2. 1(N01) , BU~a SANGA~b
 1807: 2. 1(N01)# , SANGA~b#
 2687: 2. 3(N01)# [...] , SANGA~b# KUR@g~a# [...]
 3280: 1. [...] 3(N01)# , SANGA~b
 5564: 3. [...] 1(N01) , SANGA~b#
10490: 1. 2(N14)#? , SANGA~b
10622: 1. , SANGA~c GIR3~b DUB~c
12763: 2. 1(N52)# , SANGA~b
14583: 3'. , SANGA~b? IB~b 
18084: 2. 1(N34) , SANGA~c 1(N58)


### Frequency list

TF has a standard function that generates a frequency list for any feature of your choice.

Let's do the modifiers (`@`*mod*):

In [35]:
F.modifier.freqList()

(('g', 648), ('t', 251), ('n', 39), ('r', 6), ('s', 4), ('c', 1), ('v', 1))

The result is a tuple of tuples (*modifier*, *amount of occurrences*), ordered in decreasing frequency.
We can print the list a bit more neatly as follows:

In [36]:
for (value, amount) in F.modifier.freqList():
    print(f'{amount:>4} x {value}')

 648 x g
 251 x t
  39 x n
   6 x r
   4 x s
   1 x c
   1 x v


Note that you can call this for any feature.
So let us do it for graphemes. Note that we get a frequency list for the bare graphemes, ignoring
variants and modifiers. So maybe this is not that useful.

Before we print out the complete list, let's make sure the list is not forbiddingly long.
Or rather, let us print only the top twenty.

In [37]:
for (value, amount) in F.grapheme.freqList()[0:20]:
    print(f'{amount:>4} x {value}')

29618 x …
21676 x N01
17128 x 
6956 x X
5924 x N14
1970 x EN
1846 x N57
1835 x N34
1349 x SZE
1241 x GAL
1125 x DUG
1069 x AN
1046 x U4
 892 x NUN
 881 x SAL
 879 x PAP
 877 x E2
 875 x GI
 788 x BA
 747 x SANGA


## Structures

Some node types represent things with a nested structure. We have seen *quads* above,
an other example are *clusters* of quads. Clusters are called *brackets* in the
[Oracc primer](http://oracc.museum.upenn.edu/doc/help/editinginatf/primer/inlinetutorial/index.html).

In TF, we have two mechanisms to specify that something is a part of an other thing.
One we have already encountered: `L.d(`*node*`)`, which gives you the nodes of all things whose signs
are contained in the signs belonging to *node*.

But for some purposes this is to coarse.
Consider the complex quad we encountered before:

```
|SZU2.((HI+1(N57))+(HI+1(N57)))|
```

This quad has sub-quads `SZU2`, `(HI+1(N57))+(HI+1(N57))`, and the last one
has sub-quads `HI+1(N57)` and `HI+1(N57)`, which have in turn sub-quads `HI` and `1(N57)`.

If we only look at embedding, it is a bit awkward to produce the sub-quads per level, because
if we ask for all sub-quads embedded in the outer quad, we'll get all sub-quads mentioned, regardless
of level. 

So we need an extra device, and TF offers a very generic one for this: *edge features*.

Whereas the *features* we have seen so far, such as `otype`, `grapheme`, and `variant`
annotate nodes, there are also features that annotate pairs of nodes.
A pair of nodes is called an *edge*, and indicates that there is a relationship between nodes.
Edge features give these relationships a name, and can annotate the relationships as well.

We could use edge features to relate lines to other, similar lines in the corpus, and annotate it
with the degree of similarity.

In the case of quads and clusters, we have generated edges in TF between clusters and their components,
and between quads and their components.

### Quads

The components of quads are either sub-quads or signs.
Sub-quads are also quads in TF, and they are always a composition.
Whenever a member of a sub-quad is no longer a composition, it is a *sign*.

Let's try to unravel the structure of this example quad by TF means.

#### Getting a big quad
First we need to get the node of this quad. Above we have seen the source code of the tablet in which
it occurs, from that we can pick the node of the case it is in:

In [38]:
line = T.nodeFromSection(('P005381', 'obverse:2', '1'))
line

257531

Note, that in order to specify a column, you have to specify it as *face type*`:`*column number*

In [39]:
case = L.d(line, otype='case')[0]
F.srcLn.v(case)

'1. 3(N04) , |GISZ.TE| GAR |SZU2.((HI+1(N57))+(HI+1(N57)))| GI4~a'

In order to identify our super-quad, we list all quad nodes that are part of this case.
For every quad we list the node numbers of the signs contained in it.

In order to know what signs are contained in any given node, we use the feature `oslots`.
Like the feature `otype`, this is a standard feature that is always available in a TF dataset.

Unlike `otype`, `oslots` is an edge feature: there is an edge between every node and every slot contained in it.

Whereas you use `F` to do stuff with node features, you use `E` to do business with edge features.

And whereas you use `F.feature.v(node)` to get the feature value of a node, you use 
`E.oslots.s(node)` to get the nodes for which there is an `oslots` edge from `node` to it.

In [40]:
for node in L.d(case, otype='quad'):
    print(f'{node:>6} {E.oslots.s(node)}')

149987 (111485, 111486)
149988 (111488, 111489, 111490, 111491, 111492)
149989 (111489, 111490, 111491, 111492)
149990 (111489, 111490)
149991 (111491, 111492)


On the basis of this see what the biggest quad is.
We could have been a bit more friendly to our selves by showing the actual graphemes in the quads.

In [41]:
for node in L.d(case, otype='quad'):
    print(f'{node:>6} {" ".join(F.grapheme.v(s) for s in E.oslots.s(node))}')

149987 GISZ TE
149988 SZU2 HI N57 HI N57
149989 HI N57 HI N57
149990 HI N57
149991 HI N57


So let us get the node of the biggest quad.

In [42]:
bigQuad = sorted(
    (
        quad 
        for quad in L.d(case, otype='quad')
    ),
    key = lambda q: -len(E.oslots.s(q))
)[0]
bigQuad

149988

Lo and behold, it is precisely the big quad.

#### Unraveling a quad

Now we are going to retrieve its components by following edges.
We have made `sub` edges from quads to their components.

In order to follow the `sub` edges from a node, you use 

`E.sub.f(node)`.

You can also get the `sub` edges *to* a node:

`E.sub.t(node)`.

In [43]:
E.sub.f(bigQuad)

(111488, 149989)

Let us unravel the whole structure:

In [44]:
def unravelQuad(quad):
    if F.otype.v(quad) == 'sign':
        return F.grapheme.v(quad)
    subQuads = E.sub.f(quad)
    unraveledSubQuads = [unravelQuad(subQuad) for subQuad in subQuads]
    return f'<{", ".join(unraveledSubQuads)}>'

unravelQuad(bigQuad)

'<SZU2, <<HI, N57>, <HI, N57>>>'

Later we see how we can get the exact representation of this quad back, including the operators `.` and `+`.

#### Distribution by depth

We want to sort all complex quads by the depth of their structure.
We can use the same function that we wrote for getting the depth of a case.

Here is the depth of the big quad:

In [45]:
depthStructure(bigQuad, 'quad')

3

We want to know how many quads there are, how deeply composed they get, and how often deep compositions
occur.

This is the recipe:
we visit all quads, examine their depths, and tally the appropriate depth.

In [46]:
depthDistribution = collections.Counter()

for quad in F.otype.s('quad'):
    depth = depthStructure(quad, 'quad')
    depthDistribution[depth] += 1
    
for (depth, amount) in sorted(
    depthDistribution.items(),
    key=lambda x: (-x[1], x[0]),
):
    print(f'{amount:>5} x depth {depth}')

 3735 x depth 1
  170 x depth 2
    1 x depth 3


It turns out we had spotted the deepest quad already.
Let's finish off by looking which tablets show how many quads with depth >= 2.

The code is much like above, but now we do not tally the depth of a quad, but the tablet
it occurs in.

In [47]:
tabletQuad = collections.Counter()

for quad in F.otype.s('quad'):
    depth = depthStructure(quad, 'quad')
    if depth >= 2:
        tablet = L.u(quad, otype='tablet')[0]
        tabletQuad[tablet] += 1

Before we look at the output, let us sort the tablets found on decreasing amount of complex quads:

In [48]:
tabletQuadList = sorted(
    tabletQuad,
    key=lambda t: (-tabletQuad[t], t),
)

We are not interested in tablets with just one complex quad.

We collect the tablets by their node, but when we output the results, we
want to show the tablet's number, which is stored in the feature `catalogId`.

In [49]:
singles = 0

for tablet in tabletQuadList:
    amount = tabletQuad[tablet]
    if amount == 1:
        singles += 1
    else:
        print(f'{amount:>5} complex quads in tablet {F.catalogId.v(tablet)}')
print(f'{1:>5} complex quad  in {singles} other tablets')

    6 complex quads in tablet P212396
    6 complex quads in tablet P003520
    6 complex quads in tablet P003809
    4 complex quads in tablet P005152
    3 complex quads in tablet P003591
    3 complex quads in tablet P471693
    3 complex quads in tablet P006378
    3 complex quads in tablet P325754
    3 complex quads in tablet P235782
    3 complex quads in tablet P005323
    2 complex quads in tablet P003528
    2 complex quads in tablet P004846
    2 complex quads in tablet P004893
    2 complex quads in tablet P000440
    2 complex quads in tablet P000468
    2 complex quads in tablet P000514
    2 complex quads in tablet P000157
    2 complex quads in tablet P000441
    2 complex quads in tablet P000442
    2 complex quads in tablet P002820
    2 complex quads in tablet P003768
    2 complex quads in tablet P003613
    2 complex quads in tablet P006038
    2 complex quads in tablet P005093
    2 complex quads in tablet P005188
    2 complex quads in tablet P005317
    2 comple

And, for the record, we'll have a look at the source code of one of the tablets at the top of the list.
We have already defined a function `getSource(tablet)` above, which comes in handy here. 

In [50]:
sourceLines = getSource(tabletQuadList[0])
print('\n'.join(sourceLines))

&P212396 = CDLB 2003/4 
#atf: lang qpc 
@obverse 
1.a. 8(N01)# , GAR |U4x2(N57)| AB~a 
1.a. 8(N01)# , GAR |U4x2(N57)| AB~a 
1.b. 1(N28) , |U4x1(N57)| SZE~a 
1.c. 2(N39~a) 1(N24) , |U4.1(N14)| SZE~a 
1.d1. 1(N01)# [...] , [...] SZE~a 
1.d2. 4(N14)# , X [...] 
2.a. 1(N14) , |U4x1(N01)| |LAGAB~axSZITA~a1| PAP~a#? 
2.b. 1(N24) , |U4x1(N57)| 
2.c. 1(N04) 1(N01) 1(N41) 1(N39~a) , |U4x(1(N14).2(N01))|# [...] 
3.a. 1(N01) , X |U4x1(N01)|? GUG2#? 
3.b. 2(N41) 2(N29A~b) , |U4x(1(N14).2(N01))| 
4.a. 5(N01)#? , |U4x1(N01)|#? X 
4.b1. 2(N39~a) 1(N24) , |U4x1(N01)| 
4.b2. 1(N28)#? , SZE~a# BULUG3?# [...] 
4.c. 1(N14) 1(N01) 1(N39~a) , |U4x(1(N14).2(N01))| 
@reverse 
1.a. X? 1(N28)# , |U4x1(N57)|# SZE~a 
1.a. X? 1(N28)# , |U4x1(N57)|# SZE~a 
1.b. 2(N01)# 1(N24)# , |U4x1(N01)|# 
1.c. 4(N14)# 1(N01) 1(N39~a) , |U4x(1(N14).2(N01))|# SZE~a# 
1.d. [...] 2(N01)?# [...] , SZE~a# TAR~a 1(N30~e) 
2.a. 1(N41) , |U4x1(N01)| 
2.b. 2(N04)# 4(N41)# 2(N29~a)# , |U4x(1(N14).2(N01))| 
2.c. 1(N41) 1(N39~a) 1(N24~b) 1(

### Clusters

Clusters are groups of consecutive quads between brackets.

Clusters can be nested.
As with quads, we find the members of a cluster by following `sub` edges.

#### Kinds of clusters

In our corpus we encounter several types of brackets:

* `( )a` for proper names
* `[ ]` for uncertainty
* `< >` for supplied material.

The first thing is to get on overview of the distribution of these kinds.

In [53]:
clusterTypeDistribution = collections.Counter()

for cluster in F.otype.s('cluster'):
    typ = F.type.v(cluster)
    clusterTypeDistribution[typ] += 1

for (typ, amount) in sorted(
    clusterTypeDistribution.items(),
    key=lambda x: (-x[1], x[0]),
):
    print(f'{amount:>5} x a {typ:>8}-cluster')

32321 x a uncertain-cluster
  637 x a properName-cluster
    1 x a supplied-cluster


The conversion to TF has transformed `[...]` to a cluster of one sign with grapheme `…`.
These are trivial clusters and we want to exclude them from further analysis, so we redo the counting.

First we make a sequence of all non-trivial clusters:

In [55]:
realClusters = [
    c 
    for c in F.otype.s('cluster')
    if (
        F.type.v(c) != 'uncertain' or
        len(E.oslots.s(c)) > 1 or 
        F.grapheme.v(E.oslots.s(c)[0]) != '…'
    )
]
len(realClusters)

3385

Now we redo the same analysis, but we start with the filtered cluster sequence.

In [57]:
clusterTypeDistribution = collections.Counter()

for cluster in realClusters:
    typ = F.type.v(cluster)
    clusterTypeDistribution[typ] += 1

for (typ, amount) in sorted(
    clusterTypeDistribution.items(),
    key=lambda x: (-x[1], x[0]),
):
    print(f'{amount:>5} x a {typ:>8}-cluster')

 2747 x a uncertain-cluster
  637 x a properName-cluster
    1 x a supplied-cluster


#### Lengths of clusters

How long are clusters in general?
There are two possible ways to measure the length of a cluster:

* the amount of signs it occupies;
* the amount of top-level members it has (quads or signs)

By now, the pattern to answer questions like this is becoming familiar.

We express the logic in a function, that takes the way of measuring
as a parameter.
In that way, we can easily provide a cluster-length distribution based
on measurements in signs and in quads.

In [58]:
def computeDistribution(nodes, measure):
    distribution = collections.Counter()

    for node in nodes:
        m = measure(node)
        distribution[m] += 1

    for (m, amount) in sorted(
        distribution.items(),
        key=lambda x: (-x[1], x[0]),
    ):
        print(f'{amount:>5} x a measure of {m:>8}')

In [59]:
def lengthInSigns(node):
    return len(L.d(node, otype='sign'))

def lengthInMembers(node):
    return len(E.sub.f(node))

Now we can show the length distributions of clusters by just calling `computeDistribution()`:

In [60]:
computeDistribution(realClusters, lengthInSigns)

 2691 x a measure of        1
  434 x a measure of        2
  205 x a measure of        3
   41 x a measure of        4
    9 x a measure of        5
    3 x a measure of        6
    2 x a measure of        7


In [61]:
computeDistribution(realClusters, lengthInMembers)

 2679 x a measure of        1
  452 x a measure of        2
  194 x a measure of        3
   44 x a measure of        4
   11 x a measure of        5
    4 x a measure of        6
    1 x a measure of        7


If you look closely at the code for these functions, there is nothing in it that 
is specific for clusters.

The measures are in terms of the totally generic `oslots` function, and the fairly generic
`sub` edges, which are also defined for quads.

So, in one go, we can obtain a length distribution of quads.

Note that quads can also be sub-quads.

In [62]:
computeDistribution(F.otype.s('quad'), lengthInSigns)

 3717 x a measure of        2
  181 x a measure of        3
    7 x a measure of        4
    1 x a measure of        5


In [63]:
computeDistribution(F.otype.s('quad'), lengthInMembers)

 3887 x a measure of        2
   19 x a measure of        3


## Signs

The main characteristic of a sign is its `grapheme`.
Everything we do with signs, is complicated by the fact that signs can be *augmented* with
primes, variants, modifiers and flags.

Below we show how you can get frequency lists of signs, with or without taking
augments into account.

### Writing results to file

We also want to write the results to files in your `_temp` directory, within this repo.

`writeFreqs` writes distribution data of data items called `dataName`
to a file `fileName.txt`.
In fact, it writes two files: 
* `fileName-alpha.txt`, ordered by data items
* `fileName-freq.txt`, ordered by frequency.

In [64]:
def writeFreqs(fileName, data, dataName):
    print(f'There are {len(data)} {dataName}s')

    for (sortName, sortKey) in (
        ('alpha', lambda x: (x[0], -x[1])),
        ('freq', lambda x: (-x[1], x[0])),
    ):
        with open(f'{TEMP_DIR}/{fileName}-{sortName}.txt', 'w') as fh:
            for (item, freq) in sorted(data, key=sortKey):
                if item != '':
                    fh.write(f'{freq:>5} x {item}\n')

In [65]:
writeFreqs('grapheme-plain', F.grapheme.freqList(), 'bare grapheme')

There are 667 bare graphemes


Now have a look at your TEMP_DIR and you see two generated files:

* `graphemes-plain-alpha.txt` (sorted by grapheme)
* `graphemes-plain-freq.txt` (sorted by frequency)

But we can do better, we also want the prime, variants, and modifiers taken into account.

Let us first see what they can be.

### Prime

The prime is a feature with two values: 1 or 0. 1 means: there is a prime.
Below you see how often that occurs.
Note that we count all primes here: on signs, case numbers and column numbers.

For more info and a check on the occurrences of primes, see [checks](checks.ipynb).

In [66]:
for (value, frequency) in F.prime.freqList():
    print(f'{frequency:>5} x {value}')

 5184 x 1


### Variant

The variant or *allograph* is what occurs after the grapheme and after the `~` symbol, which should be digits and/or
lowercase letters except the `x`.

Here is the frequency list of variant values.

In [67]:
for (value, frequency) in F.variant.freqList():
    print(f'{frequency:>5} x {value}')

23804 x a
 4172 x b
 1532 x c
 1356 x a1
  703 x b1
  194 x a2
  187 x d
  127 x b2
   85 x f
   73 x a3
   40 x e
   29 x c2
   22 x c1
   22 x c3
   17 x v
   14 x c5
   13 x b3
   12 x a0
   12 x d1
   11 x c4
    6 x a4
    6 x g
    5 x d2
    4 x d4
    4 x h
    2 x 3a
    2 x d3
    1 x h2


### Modifier

The modifier is what occurs after the grapheme and after the `@` symbol, which should be digits and/or
lowercase letters except the `x`.

Sometimes modifiers occur inside a repeat, then we have stored the modifier in the feature
*modifierInner*, as in

```
7(N34@f)
```

Here is the frequency list of *modifier* and *modifierInner* values.

In [68]:
for (value, frequency) in F.modifier.freqList():
    print(f'{frequency:>5} x {value}')

  648 x g
  251 x t
   39 x n
    6 x r
    4 x s
    1 x c
    1 x v


In [69]:
for (value, frequency) in F.modifierInner.freqList():
    print(f'{frequency:>5} x {value}')

   25 x f
   15 x t
    1 x r
    1 x v


### Full grapheme overview

We make a frequency list of all full graphemes, i.e. the grapheme including variant, modifier, and prime.
We show them as they appear in transcriptions.

First we show on what node types primes, variants and modifiers occur.
We only deal with cases where they occur on signs, ignoring the cases where they occur on (sub)quads.

In [70]:
for feature in ('prime', 'variant', 'modifier'):
    nodeTypes = collections.Counter()
    for n in N():
        if Fs(feature).v(n):
            nodeTypes[F.otype.v(n)] += 1
    for (value, frequency) in nodeTypes.items():
        print(f'{feature:<10}: {frequency:>5} x {value}')

prime     :  4652 x case
prime     :   523 x column
prime     :     9 x sign
variant   : 32455 x sign
modifier  :   950 x sign


Now the write out the full grapheme distribution.
Note that we use a new function, `atfFromSign`. We discuss this one and friends in the next section.

In [71]:
fullGraphemes = collections.Counter()

for n in F.otype.s('sign'):
    fullGrapheme = CUNEI.atfFromSign(n)
    fullGraphemes[fullGrapheme] += 1
    
for (value, frequency) in sorted(fullGraphemes.items(), key=lambda x: (-x[1], x[0]))[0:20]:
    print(f'{frequency:>5} x {value}')
    
writeFreqs('grapheme-full', fullGraphemes.items(), 'full grapheme')

29618 x ...
17128 x 
12996 x 1(N01)
 6956 x X
 3081 x 2(N01)
 2606 x 1(N14)
 1849 x EN~a
 1603 x 3(N01)
 1357 x 2(N14)
 1308 x SZE~a
 1304 x 5(N01)
 1224 x GAL~a
 1119 x 4(N01)
 1069 x AN
 1045 x U4
 1001 x 1(N34)
  881 x SAL
  874 x GI
  854 x PAP~a
  801 x 1(N57)
There are 1529 full graphemes


Have a look at your `_temp` directory.

## Representation

For line based concepts, such as tablet, face, column, case, comment, you can get the source
material by requesting the feature *srcLn*.

For inline concepts, such as clusters, quads, and signs, there is no such feature, so one has
to wind down the TF data and produce the right ASCII characters.
However, it is not completely trivial to generate ATF representations for those entities.
Several subtleties are involved.

In order to free you (and myself) from spelling out the logic over and over again, I have bundled a
few functions that generate ATF representations from TF data:

* `atfFromSign(sign, flags=False)`

  Returns the ATF representation for a sign, including primes, repeats, variants, modifiers,
  and, optionally, flags.


* `atfFromQuad(quad, flags=False)`

  Returns the ATF representation for a quad, including variants, modifiers,
  and, optionally, flags. For the signs belonging to this quad, the
  `flags` argument will determine whether the flags will be represented.

* `atfFromCluster(cluster)`

  Returns the ATF representation for a cluster, including the brackets.
  All quads belonging to the cluster will be represented with flags.
  Nested clusters will also be represented.

These functions have been added to Text-Fabric itself, as a submodule.
If you have installed TF (newest version), you have to say in the beginning:

```python
from tf.cunei import Cunei
```

And then after you have loaded the TF-API (`api = ...`):

```python
CUNEI = Cunei(api)
```

And then you can use the functions:

```python
print(CUNEI.atfFromQuad(q)
```

Let us pick a sign, a quad, and a cluster to represent.
We pick them from our example tablet P005138.

In [72]:
line = T.nodeFromSection(('P005381', 'obverse:2', '1'))
case = L.d(line, otype='case')[0]
F.srcLn.v(case)

'1. 3(N04) , |GISZ.TE| GAR |SZU2.((HI+1(N57))+(HI+1(N57)))| GI4~a'

Above, we stored the big quad in the variable `bigQuad`. 
Let's see what `atfFromQuad` will get from this.

In [73]:
CUNEI.atfFromQuad(bigQuad)

'|SZU2.((HI+1(N57))+(HI+1(N57)))|'

Spot on!

In P000736 there is a nice quad to show off with.

In [74]:
line = T.nodeFromSection(('P000736', 'obverse:2', '3'))
case = [
    case
    for case in L.d(line, otype='case')
    if F.fullNumber.v(case) == "3"
][0]
F.srcLn.v(case)

'3. 1(N14) , |(SZAxHI@g~a)~b|#?'

We want the big quad.
The first thing, `1(N14)`, is a simple sign, not a quad, so we need the first quad in the case.

In [75]:
quad = L.d(case, otype='quad')[0]
CUNEI.atfFromQuad(quad)

'|(SZAxHI@g~a)~b|'

Right. Now with flags:

In [76]:
CUNEI.atfFromQuad(quad, flags=True)

'|(SZAxHI@g~a)~b|#?'

Now a cluster. Let's pick one with a cluster nested in it.

In [77]:
line = T.nodeFromSection(('P471695', 'obverse:1', '2'))
case = [
    case
    for case in L.d(line, otype='case')
    if F.fullNumber.v(case) == "2b4'"
][0]
F.srcLn.v(case)

"2.b4'. , (IDIGNA [...])a "

Let's pick the outer cluster from this case. 
Remember that TF has order nodes so that nodes that have earlier slots
come before nodes that have later slots.

In [78]:
bigCluster = L.d(case, otype='cluster')[0]
CUNEI.atfFromCluster(bigCluster)

'(IDIGNA [...])a'

## Collation

We end this tutorial with a cliff hanger: collation.

For the study of cuneiform corpora it is useful to know which signs co-occur: on tablets and faces,
in columns and lines, and in cases.

Here we just show how to compute collation information with respect to tablets.

We refer to a future notebook [collation](collation.ipynb) that will be dedicated to the art and craft
of collation.

### TF ad
Already the task of computing collation of signs for tablets shows a typical pattern in the modus operandi of Text-Fabric. In order to compute collation efficiently, we have to grab a significant swath of the data, and reorganise it before we can do business.

**The bad news is**: Text-Fabric does not have the right organization for this particular problem.

**The good news is**: You can put the data in the right order.

**The best news is**: because of the IKEA-like organization of the data in TF, you can easily
put your bits and pieces in a cart, walk outside, and stack it in new ways to your liking.
Indeed, the bit that draws the data from TF and puts it into the required form, is only
a few lines of code.

### Back to collation

This is what we do:

* we collect all pairs of signs that co-occur on a tablet
* we compute a measure of co-occurrence: 
  * closer together is better
  * more tablets with the same co-occurence is better
  
We explain the steps as we go.

### Collect sign-pairs

We want signs with primes, variants and modifiers, but without flags.

In a first round, we collect all pairs of signs that have a co-occurrence on a tablet.

Suppose two signs co-occur on a tablet.
Both may have multiple occurrences.

The question is: what is a sensible measure for the the degree of co-occurrence of that pair on
that tablet?

In this tutorial we ignore the faces, columns, lines and cases that the signs occur in.
The only thing that counts is the distance between two occurrences, seen as slots.
Every sign has a sequence number, its slot number, which tells you where the sign stands in the whole
corpus. The distance between two slots is just the difference of those slots as numbers.

The distance between two signs on a tablet is the minumum distance you can find between an occurence
of the one and an occurrence of the other.

In fact, we turn distance into closeness.
If, on a tablet of 200 signs long, there are signs with occurrences on 40 and 60,
their distance is 20, but there closeness is 200 - 20 = 180.
We shall make that closeness proportional to the length of the tablet (in signs): 
180 / 200 = 0.9

The same signs may co-occur on other tablet. We also compute the relative closeness there.
In the end, we add it all up.

So every pair of signs gets a measure that expresses the total relative closeness of its co-occurrences
on all tablets where they co-occur.

We have to visit nodes multiple times and get their atf representation,
so we do it once for all and store them.

We exclude the empty graphemes and the `…` , `X` graphemes.

In [79]:
NA = {'', '…', 'X'}

signFromNode = dict()

for tablet in F.otype.s('tablet'):
    for s in L.d(tablet, otype='sign'):
        if F.grapheme.v(s) in NA:
            continue
        signFromNode[s] = CUNEI.atfFromSign(s)
len(signFromNode)

LIMIT = 20
n = 0
for i in sorted(signFromNode):
    print(f'{i:>2} = {signFromNode[i]}')
    n += 1
    if n > LIMIT:
        break

 6 = 3(N14)
 8 = SANGA~a
22 = 3(N14)
24 = 1(N14)
25 = SUHUR
28 = 1(N01)
29 = DUG~b
30 = 1(N57)
41 = 1(N46)
42 = 2(N19)
43 = 4(N41)
44 = AB~a
45 = APIN~a
46 = NUN~a
51 = SZE~a
52 = DU
53 = NUN~a
58 = n
60 = KA~a
61 = n
62 = 2(N14)


Now we work per tablet.
First we collect the relevant sign slots in a list.

Then we loop through all distinct pairs of slots of that list, and store the difference between the slots
for each pair.
If we encounter the same pair with a smaller difference, we replace the bigger difference with the smaller one.
We end up with a dictionary that gives for each pair of signs the minimal difference.

Then we turn distance into closeness and make it proportional to the length of the tablet, for all pairs.

In [85]:
pairs = collections.Counter()

for tablet in F.otype.s('tablet'):
    slots = L.d(tablet, otype='sign')
    length = slots[-1] - slots[0]
    thesePairs = {}
    for i in range(len(slots)):
        slotI = slots[i]
        if slotI not in signFromNode:
            continue
        signI = signFromNode[slotI]
        for j in range(i + 1, len(slots)):
            slotJ = slots[j]
            if slotJ not in signFromNode:
                continue
            signJ = signFromNode[slotJ]
            if signJ == signI:
                continue
            pair = (signI, signJ) if signI < signJ else (signJ, signI)
            difference = slotJ - slotI
            oldDifference = thesePairs.get(pair, None)
            if oldDifference is None or oldDifference > difference:
                thesePairs[pair] = difference
    for ((signI, signJ), difference) in thesePairs.items():
        relativeCloseness = (length - difference) / length
        pairs[(signI, signJ)] += relativeCloseness

len(pairs)

126210

That is quicker than expected!

In [86]:
for ((signI, signJ), closeness) in sorted(pairs.items(), key=lambda x: (-x[1], x[0]))[0:100]:
    print(f'{signI:<10} <=> {signJ:<10} at closeness {closeness:>7.2f}')

1(N01)     <=> 2(N01)     at closeness  795.76
1(N01)     <=> 1(N14)     at closeness  643.09
1(N01)     <=> EN~a       at closeness  574.42
1(N14)     <=> 2(N01)     at closeness  573.26
1(N01)     <=> 3(N01)     at closeness  509.83
2(N01)     <=> 3(N01)     at closeness  440.30
1(N01)     <=> N          at closeness  434.06
1(N14)     <=> 3(N01)     at closeness  415.30
1(N01)     <=> AN         at closeness  403.51
1(N01)     <=> GAL~a      at closeness  388.05
1(N14)     <=> 5(N01)     at closeness  384.51
1(N01)     <=> 4(N01)     at closeness  374.12
1(N01)     <=> 5(N01)     at closeness  371.14
1(N14)     <=> 2(N14)     at closeness  364.15
2(N01)     <=> EN~a       at closeness  355.88
1(N01)     <=> SZE~a      at closeness  353.64
1(N01)     <=> 2(N14)     at closeness  351.48
1(N01)     <=> U4         at closeness  350.78
2(N01)     <=> 2(N14)     at closeness  343.82
1(N14)     <=> 4(N01)     at closeness  342.38
2(N01)     <=> 4(N01)     at closeness  335.39
2(N01)     <=

We print all collocations to the file
[collocations-tablet.tsv](https://github.com/Dans-labs/Nino-cunei/blob/master/reports/collocations-tablet.tsv)

In [87]:
with open(f'{REPORT_DIR}/collocations-tablet.tsv', 'w') as fh:
    fh.write(f'sign1\tsign2\tcloseness\n')
    for ((signI, signJ), closeness) in sorted(pairs.items(), key=lambda x: (-x[1], x[0])):
        fh.write(f'{signI}\t{signJ}\t{closeness:>7.2f}\n')

Finally, we show an overview of how the closeness of collocated pairs is distributed.

In [88]:
closenessDistribution = collections.Counter()
for ((signI, signJ), closeness) in pairs.items():
    closenessDistribution[int(round(closeness))] += 1

for (closeness, amount) in sorted(closenessDistribution.items()):
    print(f'{amount:>4} pairs with closeness ~ {closeness:>4}')

11785 pairs with closeness ~    0
61020 pairs with closeness ~    1
18155 pairs with closeness ~    2
8877 pairs with closeness ~    3
5382 pairs with closeness ~    4
3638 pairs with closeness ~    5
2527 pairs with closeness ~    6
1968 pairs with closeness ~    7
1571 pairs with closeness ~    8
1314 pairs with closeness ~    9
1072 pairs with closeness ~   10
 850 pairs with closeness ~   11
 753 pairs with closeness ~   12
 653 pairs with closeness ~   13
 537 pairs with closeness ~   14
 492 pairs with closeness ~   15
 417 pairs with closeness ~   16
 371 pairs with closeness ~   17
 342 pairs with closeness ~   18
 311 pairs with closeness ~   19
 256 pairs with closeness ~   20
 238 pairs with closeness ~   21
 192 pairs with closeness ~   22
 191 pairs with closeness ~   23
 211 pairs with closeness ~   24
 172 pairs with closeness ~   25
 138 pairs with closeness ~   26
 149 pairs with closeness ~   27
 131 pairs with closeness ~   28
 131 pairs with closeness ~   29
 100 pa